In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import pickle

### Import and process the data

In [2]:
mp = pd.read_csv('Data/Outputs/mushed.csv')
flights = pd.read_csv('Data/Outputs/flights.csv')
flights.drop(['Unnamed: 0', 'ItinID', 'MktID',
       'Quarter',  'OriginWac',  'DestWac', 
       'ContiguousUSA', 'NumTicketsOrdered'], axis=1, inplace=True)
flights[['departure','arrival']] = flights[['departure','arrival']].astype(np.datetime64)
mpa = mp[mp['iata'].isin(flights['Origin'].unique())]
mpa.drop(['country', 'Unnamed: 0'], inplace=True, axis=1)
ports = mpa[mpa['category']=='Airport']

/var/folders/gh/lf0mc7p936xfvjr53gs0s95m0000gn/T/ipykernel_22396/511557574.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpa.drop(['country', 'Unnamed: 0'], inplace=True, axis=1)


In [83]:
with open('Data/Outputs/new_clusters', 'rb') as file:
    clusters = pickle.load(file)    

### Search function peudocode:

   **search_func**(_from, to, dates_range_):<br>
        get direct flight if available</p>
        limit the search by _dates_range_<br>
        get label of from<br>
        get label of to<br>
        get all in label of to<br>
        get all in label of from<br>
        get all flights from _all_from_ to _all_to_ from the flights data set<br>
        return **n** cheapest combinations
    

### Search functioin

In [107]:
def search(fr, to, start, stop):    
    '''Search the flight data to find flights from origin 'fr' to destination 'to' and
       cheapest flights from and to the respecting clusters in the time range 'start'-'stop'.
        Start and stop dates are str in format YYYY-MM-DD.
        'fr' and 'to' are strings of airport abbreviations like 'LAX'. 
        Returns dict of direct and indirect flights dataframes.'''    
    # Search criteria
    start = np.datetime64(start)
    stop = np.datetime64(stop)    
    # Get direct flights
    dir_fl = flights[(flights['departure']<stop)
                     &(flights['departure']>start)
                     &(flights['Origin']==fr)
                     &(flights['Dest']==to)]
    # Get indirect flights
    all_from = clusters[fr]
    all_to = clusters[to]
    all_options = flights[(flights['departure']<stop)
                      &(flights['departure']>start)
                      &(flights.Origin.isin(all_from))
                      &(flights.Dest.isin(all_to))]
    
    return {'direct':dir_fl, 'indirect':all_options.nsmallest(20,'PricePerTicket')}

### Test the function once with arbitrary parameters.

In [108]:
res = search('BUR', 'SAT', '2020-01-30', '2020-02-12')

In [109]:
res['direct']

,AIRPORT,CITY,STATE,LATITUDE,LONGITUDE,Origin,Dest,Miles,AirlineCompany,PricePerTicket,departure,arrival
1199104,Burbank-Glendale-Pasadena,Burbank,CA,34.200619,-118.358497,BUR,SAT,1211.0,WN,348.35,2020-02-04 16:00:00,2020-02-04 18:01:00


In [110]:
res['indirect'].iloc[:5]

,AIRPORT,CITY,STATE,LATITUDE,LONGITUDE,Origin,Dest,Miles,AirlineCompany,PricePerTicket,departure,arrival
6424279,Ontario International,Ontario,CA,34.056000,-117.601194,ONT,AUS,1196.0,F9,54.0,2020-01-30 04:26:00,2020-01-30 06:25:00
6424280,Ontario International,Ontario,CA,34.056000,-117.601194,ONT,AUS,1196.0,F9,54.5,2020-01-30 23:53:00,2020-01-31 01:52:00
6424281,Ontario International,Ontario,CA,34.056000,-117.601194,ONT,AUS,1196.0,F9,58.5,2020-01-31 19:20:00,2020-01-31 21:19:00
6424282,Ontario International,Ontario,CA,34.056000,-117.601194,ONT,AUS,1196.0,F9,59.5,2020-02-01 14:48:00,2020-02-01 16:47:00
5075888,Long Beach (Daugherty ),Long Beach,CA,33.817722,-118.151611,LGB,AUS,1226.0,B6,60.0,2020-02-01 18:16:00,2020-02-01 20:18:00


#### Test if i can find cheaper alternative for more than 0.7 of flights that cost more than 200$

slice flights more than 200$ assign it to 'Search' df add a column alt_price, and AirBnB

Find alternatives 
get average price of alternatives as an alt_price at the row
if no alternative alt_price = 0
search for the room price in the area fill it in AirBnB col, else add q_price, else add 0.

In [99]:
Search = flights[flights['PricePerTicket']>200.0].nlargest(36, 'PricePerTicket')

In [100]:
Search[['alt_price','AirBnB']]=0

In [101]:
Search.reset_index(inplace = True)
Search.head()

,index,AIRPORT,CITY,STATE,LATITUDE,LONGITUDE,Origin,Dest,Miles,AirlineCompany,PricePerTicket,departure,arrival,alt_price,AirBnB
0,68438,Ted Stevens Anchorage International,Anchorage,AK,61.174320,-149.996186,ANC,DEN,2405.0,UA,1000.0,2020-10-26 15:36:00,2020-10-26 19:36:00,0,0
1,72807,Ted Stevens Anchorage International,Anchorage,AK,61.174320,-149.996186,ANC,IAH,3266.0,UA,1000.0,2020-11-24 18:34:00,2020-11-25 00:00:00,0,0
2,174269,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944,ATL,DEN,1199.0,DL,1000.0,2020-01-27 03:35:00,2020-01-27 05:34:00,0,0
3,185288,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944,ATL,DFW,731.0,AA,1000.0,2020-03-01 10:11:00,2020-03-01 11:24:00,0,0
4,404505,William B Hartsfield-Atlanta Intl,Atlanta,GA,33.640444,-84.426944,ATL,PHL,666.0,AA,1000.0,2020-10-08 08:39:00,2020-10-08 09:45:00,0,0


In [102]:
frl=Search.Origin
tol=Search.Dest

In [103]:
startl = Search['departure'].apply(lambda x: x - np.timedelta64(5, 'D'))

In [104]:
endl = Search['arrival'].apply(lambda x: x + np.timedelta64(5, 'D'))

In [105]:
import time
strat=time.time()

monster = [search(a,b,c,d)['indirect'] for a, b, c, d in zip(frl,tol,startl,endl)]

sotp = time.time()
elapsed=sotp-strat
print(elapsed)

53.38128995895386


In [106]:
len([i for i in monster if not i.empty])

22

In [111]:
with open('Data/Outputs/monster', 'wb') as file:
    pickle.dump(monster, file)

In [96]:
60**2

3600

In [112]:
22/36

0.6111111111111112